### Working with LLM SQL Queries in Databricks

### Loading CSV file into dbfs (Databricks File System)

In [ ]:
%sh
rm -r /dbfs/delta_lab
mkdir /dbfs/delta_lab
wget -O /dbfs/delta_lab/products.csv https://raw.githubusercontent.com/kuljotSB/DatabricksGenAIEngineer/refs/heads/main/GenAI_Fundamentals/restaurant_reviews.csv

### Creating the Bronze Layer Dataframe

In [ ]:
bronze_df = spark.read.load('/delta_lab/restaurant_reviews.csv', format='csv', header=True)
display(bronze_df.limit(10))

### Creating the Bronze Schema

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS YOUR_WORKSPACE_NAME.Bronze;

### Storing the Bronze Layer as a Delta Table

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Storing the Bronze Layer as a Delta Table in the Databricks File System (DBFS)
bronzeDeltaTablePath = '/delta/bronze_restaurant_reviews'
bronze_df.write.format('delta').mode('overwrite').save(bronzeDeltaTablePath)

# Storing the Bronze Layer as a Delta Table in the Data Catalog
bronze_df.write.format('delta').saveAsTable('Bronze.bronze_restaurant_reviews')

### Creating the Gold Layer by Performing Business Operations

### Loading the Gold Layer Dataframe from the Bronze Delta Table

In [ ]:
gold_df = spark.read.format('delta').table('Bronze.bronze_restaurant_reviews')
display(gold_df.limit(10))

### Using LLM to Generate Sentiment Analysis and Summarization

In [ ]:
gold_df = spark.sql("""
SELECT
    CustomerID,
    CustomerName,
    Products,
    Review,

    ai_query(
      databricks-claude-sonnet-4-5,
      "Classify the customer sentiment as Positive, Neutral, or Negative. 
       Respond with only one word.: " || Review
    ) AS sentiment,

    ai_query(
      databricks-claude-sonnet-4-5,
      "Summarize this restaurant review in one short sentence.: " || Review
    ) AS summary

FROM YOUR_UC_CATALOG_NAME.Bronze.restaurant_reviews
 """)

display(gold_df.limit(10))

### Creating the Gold Schema in Unity Catalog

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS YOUR_UC_CATALOG_NAME.Gold;

### Stroing the Gold Layer as a Delta Table in Unity Catalog

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Storing the Bronze Layer as a Delta Table in the Databricks File System (DBFS)
goldDeltaTablePath = '/delta/gold_sales_orders'
gold_df.write.format('delta').mode('overwrite').save(goldDeltaTablePath)

# Storing the Bronze Layer as a Delta Table in the Data Catalog
gold_df.write.format('delta').saveAsTable('Gold.restaurant_reviews')